In [1]:
import pandas as pd
df = pd.read_csv("spam.csv",encoding = "ISO-8859-1")

In [2]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"], axis= 1,inplace = True)

In [4]:
df.columns = ["label","mail"]

In [5]:
df.head()

,label,mail
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df["label"].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [7]:
df.loc[df["label"] == "ham", "label"] = 0
df.loc[df["label"] == "spam","label"] = 1

In [8]:
df.head()

,label,mail
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [50]:
df_spam = df[df["label"]==1]

In [51]:
df_spam.reset_index().drop("index",axis=1,inplace=True)

In [52]:
df_spam.iloc[4]["mail"]

'SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info'

In [9]:
import nltk
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amalr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
def preprocess(df):
  ps = PorterStemmer()
  corpus = []
  for i in range(len(df)):
    sent = re.sub("[^a-zA-Z]"," ",df["mail"][i])
    sent = sent.lower()
    words = sent.split()
    words = [ps.stem(word) for word in words if word not in set(stopwords.words("english"))]
    sent = " ".join(words)
    corpus.append(sent)
  return corpus

In [12]:
corpus = preprocess(df)

In [13]:
corpus[:10]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free']

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
x = cv.fit_transform(corpus).toarray()


In [33]:
file = "cv.sav"
pickle.dump(cv, open(file, 'wb'))


In [15]:
x.shape

(5572, 5000)

In [16]:
y = list(df["label"])

In [17]:
len(y)

5572

In [18]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,
                                                 test_size = 0.25,
                                                 random_state =  0)

In [19]:
#print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [20]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

In [21]:
model.fit(x_train,y_train)

MultinomialNB()

In [41]:
y_pred = model.predict(x_test)

In [42]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print("The accuracy of the model is ",accuracy_score(y_test,y_pred))
print("\nThe Confusion matrix: ")
print(confusion_matrix(y_test,y_pred))
print("\nThe classification Report :")
print(classification_report(y_test,y_pred))

The accuracy of the model is  0.9827709978463748

The Confusion matrix: 
[[1178   18]
 [   6  191]]

The classification Report :
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1196
           1       0.91      0.97      0.94       197

    accuracy                           0.98      1393
   macro avg       0.95      0.98      0.97      1393
weighted avg       0.98      0.98      0.98      1393



In [24]:
def prepare(mail):
    mails = [mail]
    ps = PorterStemmer()
    corpus1 = []
    for i in mails:

        sent = re.sub("[^a-zA-Z]"," ",i)
        sent = sent.lower()
        words = sent.split()
        words = [ps.stem(word) for word in words if word not in set(stopwords.words("english"))]
        sent = " ".join(words)
        corpus1.append(sent)

    x = cv.transform(corpus1).toarray()

    return x




x = prepare("Why is this message in spam? It is similar to messages that were identified as spam in the past.")
x.shape

(1, 5000)

In [25]:
model.predict(x)

array([1])

In [31]:
import pickle

In [32]:
filen = "final.sav"
pickle.dump(model,open(filen,"wb"))